In [17]:
import numpy as np
import math

eps=1e-6

In [18]:
# First question

In [19]:
def sdp_solver(C,A,b):
    # We're searching for a strictly feasible point
    m,n,_ = A.shape
    mat3d=np.zeros((1,n,n))
    mat3d[0:,:,]=np.identity(n)
    new_A = np.concatenate((A,mat3d))
    new_b=np.zeros(m+1)
    new_b[m]=1
    liste = [0 for i in range (m)]+[-min(np.linalg.eigh(C)[0])+1]
    mu = barrier_method(C,new_A,new_b,liste,stop=True)[1]
    
    if mu[m]>=0:
        print("No feasible point")
        return False
    
    # If we found it, let's start!
    else:
        #print("We found a strictly feasible point, we start the resolution",'\n')
        return barrier_method(C,A,b,mu[0:m],stop=False)

def F(C, A, mu):
    m,n,_ = A.shape
    tmp = np.zeros((n,n))
    for i in range(m):
        tmp += np.dot(mu[i],A[i])
    return tmp + C

def f_t(t, C, A, b, mu):
    F_mu = F(C, A, mu)
    if min(np.linalg.eigh(F_mu)[0]) <= 0:
        return math.inf
    else:
        return t * np.array(np.dot(mu,np.transpose(b))) - np.log(np.linalg.det(F_mu))
    
def nabla_f_t(t, C, A, b, mu):
    m,n,_ = A.shape
    v = [0 for i in range (m)]
    inv_F_mu = np.linalg.inv(F(C, A, mu))
    for i in range (m):
        v[i] = np.trace(inv_F_mu * A[i, :, :])
    return t*np.array(b) - v

def nabla2_f_t(t, C, A, b, mu):
    m,n,_ = A.shape
    h = np.zeros((m,m))
    inv_F_mu = np.linalg.inv(F(C, A, mu))
    for i in range(m):
        for j in range(m):
            h[i, j] = np.trace(np.dot(inv_F_mu,np.dot(A[i, :, :],np.dot(inv_F_mu, A[j, :, :]))))
    return h

def barrier_method(C, A, b, mu_0,stop):
    m,n,_ = A.shape
    mu = mu_0
    gamma = 15
    t = 10
    while True:
        mu = newton(t, C, A, b, mu,stop=stop)
        if n / t < eps:
            break
        if stop and mu[m-1]<0:
            break
        t = gamma * t
    return (1 / t * np.linalg.inv(F(C, A, mu)), mu)


def newton(t, C, A, b, x_0,stop):
    m,n,_ = A.shape
    x = x_0
    while True:
        nabla_f_t_x = nabla_f_t(t, C, A, b, x)
        nabla2_f_t_x = nabla2_f_t(t, C, A, b, x)
        inv_nabla2_f_t_x = np.linalg.inv(nabla2_f_t_x)   
        delta_x = (- np.dot(nabla_f_t_x,inv_nabla2_f_t_x)).tolist()
        if len(delta_x)==1:
            delta_x=delta_x[0]
        lambda2 = (- np.dot(delta_x,nabla_f_t_x.T))
        
        if lambda2 / 2 < eps:
            break
        if stop and x[m-1]<0:
            break
        fact = line_search(t, C, A, b, x, delta_x)
        x += np.dot(fact, delta_x)
    return x

                     
def line_search(t, C, A, b, x, delta_x):
    alpha = 0.01
    beta = 0.5
    fact = 1
    nabla_f_t_x = nabla_f_t(t, C, A, b, x)
    f_t_x = f_t(t, C, A, b, x)
    while f_t(t, C, A, b, x + np.dot(delta_x,fact)) > f_t_x + (alpha * fact * np.dot(delta_x,nabla_f_t_x.T)):
        fact = beta * fact
    return fact

In [20]:
# Generator of random SDP

In [21]:
def random_SDP(boule=True):
    n,m = np.random.randint(2,10),np.random.randint(2,10)
    if boule==False:
        m=1
    C = np.matrix(np.random.randint(0,100,size=(n,n)))
    C = C*C.T
    A = np.zeros((m,n,n))
    for i in range(m):
        TMP = np.random.randint(0,100,size=(n,n))
        A[i:,:,] = TMP * TMP.T
    b=np.random.randint(0,100,size=(1,m))
    return C,A,b

In [22]:
#turn "Test" to "True" to test it on random samples
test = True

if test:
    print("The general form of a SDP is :")
    print("Minimize Tr(C*X)")
    print("s.t. Tr(A_i*X)=b_i, i=1...m")
    print("     X >= 0 \n \n")
    for i in range(4):
        C,A,b=random_SDP()
        print("This (random) SDP is such that :")
        print("C=",C)
        print("A_1...A_m=",A)
        print("b=",b)
        (x, mu) = sdp_solver(C, A, b)
        x=np.matrix(x)
        print("And its solution is:\n")
        print(np.trace(C * x),'\n')

The general form of a SDP is :
Minimize Tr(C*X)
s.t. Tr(A_i*X)=b_i, i=1...m
     X >= 0 
 

This (random) SDP is such that :
C= [[3280 1984]
 [1984 1709]]
A_1...A_m= [[[2704.  552.]
  [ 552. 6724.]]

 [[4761.  252.]
  [ 252. 7921.]]

 [[ 529.    0.]
  [   0.  676.]]

 [[8649.   90.]
  [  90. 3136.]]

 [[3721. 4664.]
  [4664. 2209.]]

 [[7396. 1080.]
  [1080. 8100.]]

 [[3844. 2583.]
  [2583. 7396.]]

 [[9409. 3380.]
  [3380.  576.]]

 [[ 100. 3069.]
  [3069.  196.]]]
b= [[25 87 90 91 50 79  0 24 93]]
And its solution is:

8.503112025596655e-07 

This (random) SDP is such that :
C= [[30859 25244 31449 13754 17833 22953 16636]
 [25244 40180 39290 16418 22297 23471 27317]
 [31449 39290 45422 20744 25657 30473 29608]
 [13754 16418 20744 12165  9473 13805 10768]
 [17833 22297 25657  9473 22507 21265 24699]
 [22953 23471 30473 13805 21265 26105 22941]
 [16636 27317 29608 10768 24699 22941 29299]]
A_1...A_m= [[[3.969e+03 2.451e+03 4.898e+03 1.197e+03 2.666e+03 1.562e+03 4.960e+02]
  [2.451e+0

In [23]:
# Second question

In [24]:
def one_constraint(C, A, M):
    n,_ = M.shape
    val_propres = np.linalg.eigh(M)[0]
    rang = 0
    for i in range(n):
        if val_propres[i] > eps:
            rang = rang + 1
    if rang == 0:
        return np.zeros(n)
    elif rang == 1:
        tmp = np.linalg.eigh(M)
        val = tmp[0][0]
        v = np.sqrt(val) * tmp[1][:, 0]
        return v
    elif rang == 2:
        tmp = np.linalg.eig(M)
        k1 = np.concatenate((np.array(tmp[1][:, 0 : 2]), np.zeros((n, n - 2))),axis=1)
        k21 = [ [np.sqrt(tmp[0][0]),0],[0,np.sqrt(tmp[0][1])] ]
        k21 = np.concatenate((k21,np.zeros((n-2,2))))
        V = np.dot(k1,k21)
        (lamb, P) = np.linalg.eigh(V.T * C * V)
        V = np.dot(V, P)
        s_g = V.T * A * V
        sig = [s_g[0, 0], s_g[1, 1]]
        gamma = s_g[0, 1]
        mat = [lamb.T, np.transpose(sig)]
        if -eps < np.linalg.det(mat) < eps:
            w = [sum(lamb), sum(sig)]
            if w[0] * lamb[0] > 0:
                alpha = np.sqrt(w[0] / lamb[0])
                return alpha * V[:, 0]
            else:
                beta = np.sqrt(w[1] / lamb[1])
                return beta * V[:, 1]
        else:
            z = np.linalg.inv(mat) * [0, gamma]
            delta = 4 * (z[1] - z[0])^2 + 4
            roots = [ (-2*(z[1]-z[0])+np.sqrt(delta))/2, (-2*(z[1]-z[0])-np.sqrt(delta))/2 ]
            t = 0
            if 1 + 2 * roots[0] * z[0] > 0:
                t = roots[0]
            else:
                t = roots[1]
            alpha = 1 / np.sqrt(1 + 2 * t * z[0])
            beta = t * alpha
            return alpha * V[:,0] + beta*V[:, 1]
    else: # rank > 2
        tmp = np.linalg.eig(M) #random eigenvector of M
        val = tmp[0][0]
        print(val,"\n")
        v = np.sqrt(val) * tmp[1][:, 0]
        new_v = one_constraint(C, A, M - v * v.T)
        return one_constraint(C, A, np.dot(v,v.T) + np.dot(new_v,new_v.T))


In [26]:
test_one_constraint=True

if test_one_constraint:
    print("The general form of a SDP is :")
    print("Minimize Tr(C*X)")
    print("s.t. Tr(A_i*X)=b_i, i=1...m")
    print("     X >= 0 \n \n")
    for i in range(4):
        C,A,b=random_SDP(boule=False)
        print("This (random) SDP is such that :")
        print("C=",C)
        print("A_1...A_m=",A)
        print("b=",b)
        (x, mu) = sdp_solver(C, A, b)
        x=np.matrix(x)
        print("And its solution is:\n")
        print(np.trace(C * x),'\n')
        v = one_constraint(C, A[0, :, :], x)
        v=np.matrix(v)
        print("And the solution under the form x^T*x is")
        print(np.trace(C * v * v.T))
        print("\n")

The general form of a SDP is :
Minimize Tr(C*X)
s.t. Tr(A_i*X)=b_i, i=1...m
     X >= 0 
 

This (random) SDP is such that :
C= [[18420  8656 14943  8831 14603 11318]
 [ 8656  9014  6608  5734  8295  6413]
 [14943  6608 13449  8946 13205 10981]
 [ 8831  5734  8946  9904 11122 10491]
 [14603  8295 13205 11122 16859 14814]
 [11318  6413 10981 10491 14814 15674]]
A_1...A_m= [[[  49.  864. 5056.  456. 2775. 2842.]
  [ 864.  441.  344. 3237. 5740. 2352.]
  [5056.  344. 7744. 7905. 1708. 4489.]
  [ 456. 3237. 7905. 1225. 9021.  688.]
  [2775. 5740. 1708. 9021. 7569. 1806.]
  [2842. 2352. 4489.  688. 1806. 1089.]]]
b= [[21]]
And its solution is:

3.071475324609905 

And the solution under the form x^T*x is
1.3735443107478565e-07


This (random) SDP is such that :
C= [[17759 14593 14020 18309 20073 11661 21822 20472]
 [14593 14883 17470 19712 17638 11811 19399 18826]
 [14020 17470 32407 28418 19143 21484 23705 24440]
 [18309 19712 28418 32156 24064 19575 30513 26012]
 [20073 17638 19143 24064 